----

*Problemas:*

- Os arquivos `[DGroh,MRocha,ALeles]` estão faltando muitas linhas... vai gerar uma inconsistência muito forte, visto que só terão 2 arquivos "preenchidos"

- Os arquivos `[Viní16,LCoelho]` possuem muita inconsistência (valores acima de 10), o `Viní16` inclusive, possui linhas que **não** podem existir, vide "10" em na competência _segurança_ 

# Análise sobre o perfil dos alunos formandos/formados do curso de Engenharia de Computação

**Resumo**

Este arquivo irá conter todos os dados referentes à pesquisa realizada.

----


### Imports

In [4]:
# Módulo usado para obter a grade
from modules.Scrapping import get_grid
# Realiza o plot do grafo/grade
from modules.Plot import weighted_graph, only_grid
# Dataframe
import pandas as pd
# Lista os arquivos no diretório
from os import listdir
# Bitwise operation em vetores
import numpy as np

# Graph
import networkx as nx

### Funções

In [5]:
def grid(grid):
    GraphPre, GraphCo = get_grid(grid)
    # Print grid
    # only_grid(GraphPre.nodes(), GraphPre.edges(), GraphCo.edges())
    return GraphPre, GraphCo


def read_csv():
    # Get form's infos
    infos = []
    sheetsDir = "../assets/sheets"
    for file in listdir(sheetsDir):
        tmpDF = pd.read_csv(f"{sheetsDir}/{file}").fillna(0)
        # Remove two last lines
        tmpDF = tmpDF[:-2]
        # Renaming columns
        header = tmpDF.columns.to_list()
        header[0] = 'Periodo'
        header[1] = 'Sigla'
        header[2] = 'Nome da disciplina'
        header[-1] = 'Somatório'
        tmpDF.columns = header
        # Removing unecessary columns
        tmpDF.drop(['Periodo', 'Nome da disciplina',
                    'Somatório'], axis=1, inplace=True)
        # Turn initials into index
        tmpDF.set_index('Sigla', inplace=True)

        # Append this dataframe to info vec
        infos.append(tmpDF)

    # Return a list of dataframes
    return infos

### Obtêm o grafo da grade e gera a visualização dela

In [6]:
pre, co = grid('0192015')

### Obtêm os pesos dos arquivos csv

In [7]:
infos = read_csv()

### Realizando a média dos arquivos

In [8]:
# Juntando os arquivos num único dataframe
out = pd.concat(infos)

# Realizando 
out = out.groupby(out.index)
out = out.mean()

## Transformando a grade em uma relação de competências

- Cada competência gera um único _grafo_, assim, é necessário propagar os nós para todos os grafos, e atribuir os vértices de acordo com a entrada dos alunos

In [9]:
# Obtendo uma cópia do grafo de pré requisito (este grafo possui o período de cada disciplina)
nodes = pre.copy()

# Removendo os vértices (conexões, que nates eram os pré requisitos)
nodes.remove_edges_from(list(nodes.edges()))

In [142]:
# Obtêndo o nome de cada coluna (grafo novo)
competencias = out.columns.to_list()

In [11]:
# Removendo os nós que não constam naquela planilha
nos_para_remocao = [no for no in list(nodes.nodes()) if no not in out.index.to_list()]
nodes.remove_nodes_from(nos_para_remocao)

In [12]:
# Variável que irá iterar sobre os períodos
def get_periodo():
    """
    Função que computa o período atual
    """
    p = 0
    while p<10:
        p+= 1
        yield p

In [118]:
def debug(msg,*args):
    """
    Print a debug message

    Parameters
    ----------
    msg: string
    """
    printString = msg
    if args:
        vec = list(map(lambda x: str(x), list(args)))
        vec = ', '.join(vec)
        printString += ': ' + vec
    print(f'[DEBUG] -> {printString}')



In [138]:
def get_materias(periodo):
    """
    Obtêm todas a lista de todas as siglas de matérias do período

    Parameters
    ----------
    periodo: número que indica o período atual entre [1,10]

    Returns
    -------
    Uma lista contendo todas as siglas das matérias deste período
    """
    global nodes
    return [sigla for sigla,data in nodes.nodes(data=True) if data['period']==str(periodo)]



def check_competencia(materias_atuais, competencia):
    """
    Verifica se tem algum valor para esta competência neste periodo

    Parameters
    ----------
    materias_atuais: lista de materias no periodo atual
    competencia: competencia a ser analizada

    Returns
    -------
    True para caso não tenha uma disciplina no período atual que possua esta competência
    """
    global out
    return sum([out.loc[i][competencia] for i in materias_atuais]) == 0



def get_peso_competencia(materias_atuais, competencia, materia):
    """
    FIXME: Peso errado? Não, o peso das arestas é referente ao período do vértice de onde sai, se só tiver 1 matéria, irá ser peso 1 para qualquer aresta.

    Obtêm o peso da disciplina normalizado pelo peso de disciplinas no semestre, i.e., o máximo que uma competência pode ter no semestre é 1 ou 100%

    Parameters
    ----------
    materias_atuais: lista de materias no periodo atual
    competencia: competencia a ser analizada
    materia: materia que se deseja obter o peso da competencia para este periodo

    Returns
    -------
    Um float arredondado à duas casas decimais
    """
    global out
    total = sum([out.loc[i][competencia] for i in materias_atuais])

    # debug(f'get_peso_competencia({materia},  {materias_atuais}) -> Total {total}, Matéria {out.loc[materia][competencia]}\n')
    return round(out.loc[materia][competencia]/total, 2)

In [143]:
# Instância que irá ter todos os objetos de grafos indexados pelas competências
grafos = {}

# Obtêm as matérias do período seguinte (ou do próximo) que tiver este requisito, gerando conexões entre essas disciplinas
for competencia in competencias:
    # Instancia um novo grafo (baseado no grafo que já possui os nós) para essa competência
    grafos[competencia] = nodes.copy()

     # Itera essa competência para os próximos períodos
    for periodo in get_periodo():
        # Obtêm a lista de todas as matérias do período atual
        materias_periodo_atual = get_materias(periodo)
        # Se maior igual à zero, nenhuma das matérias do período possui valor para essa competência
        if check_competencia(materias_periodo_atual,competencia):
            # portanto, pode pular esta disciplina
            continue

        # Lista para armazenar todos os nós de matérias que possuem o mesmo requisito nos próximos períodos
        materias_herdeiras = []

        # Itera sobre os próximos períodos
        prox_periodo = periodo
        while prox_periodo < 10:
            prox_periodo += 1
            # Obtêm a lista de todas as matérias do período atual
            materias_proximo_periodo = get_materias(prox_periodo)
            # Verifica se neste período possui alguma matéria que herda aquela competência
            if check_competencia(materias_proximo_periodo, competencia):
                continue
            else:
                # Caso encontre alguma matéria neste período coloca ela na lista de materias que irão herdar a competência
                for i in materias_proximo_periodo:
                    if out.loc[i][competencia] > 0:
                        materias_herdeiras.append(i)
                # Força a saída deste loop
                break

        # Para cada matéria atual que possui a competência e cada matéria futura que irá herdar, realiza a "junção"
        for materia_atual in materias_periodo_atual:
            # Verifica se a matéria possui esta competência
            if not out.loc[materia_atual][competencia]:
                continue
            for materia_herdeira in materias_herdeiras:
                peso = get_peso_competencia(materias_periodo_atual,competencia,materia_atual)
                grafos[competencia].add_edge(
                    materia_atual,
                    materia_herdeira, 
                    weight=peso
                )



In [144]:
# Itera sobre cada grafo de competencia
for competencia in grafos.keys():
    # Gera um plot para cada um com o peso saindo de cada matéria
    weighted_graph(
        grafos[competencia].nodes(),
        list(grafos[competencia].edges(data=True)),
        competencia.title().replace(' ','').replace('.','')
    )


# ToDO:
- [x] Gerar uma visualização de grafo para cada competência (Próxima reunião)
- [ ] Com isso, basta verificar para cada disciplina o valor que o aluno obteve e atrelar isso ao valor da competência que será propagado. Realizar uma média de valor por périodo antes, atribuir uma nota